In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import gdown

In [2]:
# Loading data from GDrive.
file_id = "110IwUQC6J0RoDpJyC1SqBTzvGr-qQKua"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
df = pd.read_csv(file_extension, sep=",")

Downloading...
From (original): https://drive.google.com/uc?id=110IwUQC6J0RoDpJyC1SqBTzvGr-qQKua
From (redirected): https://drive.google.com/uc?id=110IwUQC6J0RoDpJyC1SqBTzvGr-qQKua&confirm=t&uuid=bcf4562f-0820-4a02-b924-1109b3ef0162
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 252M/252M [00:08<00:00, 28.8MB/s] 


In [3]:
df.head()

,ad_id,result,job_field,position_category,position,education_level,work_time,vacancies,city,institution,...,date_announced,views,year_announced,promocja,kandydaci,salary_for_position,offer_salary,women_percentage,fluctuation,employment_contract
0,130994,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,0.5,1,Olesno,Powiatowy Inspektorat Weterynarii w Oleśnie,...,2023-12-02,183.0,2023,1.0,1.0,NaN,nie podano wynagrodzenia,0.666667,0.064683,1.19
1,130970,0,tech/construction,stanowiska samodzielne,starszy specjalista,3,1.0,1,Bydgoszcz,Wojewódzki Inspektorat Nadzoru Budowlanego w B...,...,2023-12-01,296.0,2023,0.0,4.0,NaN,nie podano wynagrodzenia,0.580645,0.036010,0.50
2,130898,1,vet,stanowiska samodzielne,asystent,2,1.0,1,Chrzanów,Powiatowy Inspektorat Weterynarii w Chrzanowie,...,2023-11-30,701.0,2023,0.0,18.0,NaN,"4400,00 zł brutto",0.833333,0.101215,NaN
3,130919,0,uniformed services,stanowiska samodzielne,specjalista,4,1.0,1,Warszawa,Komenda Stołeczna Policji w Warszawie,...,2023-11-30,456.0,2023,NaN,NaN,NaN,"4940,34 zł brutto",NaN,NaN,NaN
4,130837,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,1.0,1,Świdnica,Powiatowy Inspektorat Weterynarii w Świdnicy,...,2023-11-29,228.0,2023,1.0,2.0,10268.888889,nie podano wynagrodzenia,0.833333,0.000000,3.00


##### Merging with Teryt - data about location for geopandas.

In [8]:
# Loading data.
file_id = "16YMXsOSYkQLncm0nqfqw5Z7mAW8TF-ql"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
teryt = pd.read_csv(file_extension, sep=",")
teryt.head()

Downloading...
From: https://drive.google.com/uc?id=16YMXsOSYkQLncm0nqfqw5Z7mAW8TF-ql
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 1.32M/1.32M [00:00<00:00, 11.9MB/s]


,job_id,WOJ,POW
0,131450,14,65
1,131558,28,16
2,131502,14,65
3,131510,6,4
4,131408,14,65


In [7]:
# Merging data.
df_with_teryt = pd.merge(df, teryt, left_on=['ad_id'], right_on=['job_id'], how="left")

# Name changes.
df_with_teryt["voivodeship"] = df_with_teryt["WOJ"]
df_with_teryt["district"] = df_with_teryt["POW"]
df_with_teryt.drop(columns=[
    "WOJ", "POW", "job_id"
], inplace=True)
df_with_teryt.head()

,ad_id,result,job_field,position_category,position,education_level,work_time,vacancies,city,institution,...,year_announced,promocja,kandydaci,salary_for_position,offer_salary,women_percentage,fluctuation,employment_contract,voivodeship,district
0,130994,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,0.5,1,Olesno,Powiatowy Inspektorat Weterynarii w Oleśnie,...,2023,1.0,1.0,NaN,nie podano wynagrodzenia,0.666667,0.064683,1.19,16,8
1,130970,0,tech/construction,stanowiska samodzielne,starszy specjalista,3,1.0,1,Bydgoszcz,Wojewódzki Inspektorat Nadzoru Budowlanego w B...,...,2023,0.0,4.0,NaN,nie podano wynagrodzenia,0.580645,0.036010,0.50,4,61
2,130898,1,vet,stanowiska samodzielne,asystent,2,1.0,1,Chrzanów,Powiatowy Inspektorat Weterynarii w Chrzanowie,...,2023,0.0,18.0,NaN,"4400,00 zł brutto",0.833333,0.101215,NaN,12,3
3,130919,0,uniformed services,stanowiska samodzielne,specjalista,4,1.0,1,Warszawa,Komenda Stołeczna Policji w Warszawie,...,2023,NaN,NaN,NaN,"4940,34 zł brutto",NaN,NaN,NaN,14,65
4,130837,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,1.0,1,Świdnica,Powiatowy Inspektorat Weterynarii w Świdnicy,...,2023,1.0,2.0,10268.888889,nie podano wynagrodzenia,0.833333,0.000000,3.00,2,19
